# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Откроем файл с данными и изучим общую информацию

In [4]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


   После изучения общей информации можно сделать следующий вывод: в столбцах "трудовой стаж" и "доход в месяц" много пропусков, также в этих столбцах тип переменных.float64. Постараемся избавиться от пропусков и заменить тип данных.

## Предобработка данных

### Обработка пропусков

In [5]:
data[data['days_employed'].isna()]
data[data['total_income'].isna()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости


In [6]:
# заменим пропуски в столбце "доход в месяц" 
# медианой с учетом группировки по типу занятости

data['total_income'] = data['total_income'].\
fillna(data.groupby('income_type')['total_income'].transform('median'))

# заменим пропуски в столбце "стаж работы" 
# средним значением с учетом группировки по возрасту

data['days_employed'] = data['days_employed'].\
fillna(data.groupby('dob_years')['days_employed'].transform('mean'))
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Столбец "трудовой стаж" оставили без изменений, он не влияет на анализ в нашей работе.
В столбце "доход в месяц" заменили пропуски медианой, использовали fillna с учетом группировки по типу занятости. Пропуски связаны с нежеланием клиентов предоставлять о себе данные.

### Замена типа данных

In [7]:
data['days_employed'].abs()
 
data['days_employed'] = data['days_employed'].round().astype('Int64')
data['total_income'] = data['total_income'].astype(int)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null Int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: Int64(1), int64(6), object(5)
memory usage: 2.0+ MB


Мы заменили тип данных в столбцах "трудовой стаж" и "доход в месяц" с float64 на int64, а также привели значения в графе "трудовой стаж" к абсолютным.

### Обработка дубликатов

In [8]:
data.duplicated().sum()
data['education'] = data['education'].str.lower()
data = data.drop_duplicates().reset_index(drop=True) 
data.duplicated().sum()
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8438,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,-4025,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,-4125,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,-926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,-2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,-153,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,-6930,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,-2189,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


Сначала мы перевелии в нижний регистр данные из столбца "образование" с помощью str.lower(), затем с помощью duplicated().sum() нашли количество дубликатов  и с помощью drop_duplicates() избавились от них.

### Лемматизация

In [9]:
from pymystem3 import Mystem  # импорт лемматизатора
m = Mystem()

unique_purpose = data['purpose'].unique()   # найдем сначала уникальные цели
#print(unique_purpose)

lemmas_list = []

for purpose in unique_purpose:                # в цикле создаем список лемм        
    lemmas = ''.join(m.lemmatize(purpose)).strip()
    lemmas_list.append(lemmas) 

from collections import Counter               # посмотрим частотность 
print(Counter(lemmas_list))

def create_category_purpose(row):
    lem_purpose = m.lemmatize(row['purpose'])
    if 'недвижимость' in lem_purpose:
        return 'недвижимость'
    if 'жилье' in lem_purpose:
        return 'недвижимость'
    if 'автомобиль' in lem_purpose:
        return 'автомобиль'
    if 'образование' in lem_purpose:
        return 'образование'
    return 'прочие цели'
data['type_purpose'] = data.apply(create_category_purpose, axis=1)
#data.head(20)


Counter({'автомобиль': 2, 'покупка жилье': 1, 'приобретение автомобиль': 1, 'дополнительный образование': 1, 'сыграть свадьба': 1, 'операция с жилье': 1, 'образование': 1, 'на проведение свадьба': 1, 'покупка жилье для семья': 1, 'покупка недвижимость': 1, 'покупка коммерческий недвижимость': 1, 'покупка жилой недвижимость': 1, 'строительство собственный недвижимость': 1, 'недвижимость': 1, 'строительство недвижимость': 1, 'на покупка подержать автомобиль': 1, 'на покупка свой автомобиль': 1, 'операция с коммерческий недвижимость': 1, 'строительство жилой недвижимость': 1, 'жилье': 1, 'операция со свой недвижимость': 1, 'заниматься образование': 1, 'сделка с подержанный автомобиль': 1, 'получение образование': 1, 'свадьба': 1, 'получение дополнительный образование': 1, 'покупка свой жилье': 1, 'операция с недвижимость': 1, 'получение высокий образование': 1, 'свой автомобиль': 1, 'сделка с автомобиль': 1, 'профильный образование': 1, 'высокий образование': 1, 'покупка жилье для сдача':

Мы создали новый солбец "type_purpose" или "тип цели" с помощью apply. Содали функцию с конкретной целью (недвижимость, автомобиль, образование и прочие цели). Таким образом, лемматизировали цели.

### Категоризация данных

In [10]:
# устраняем ошибки в графе "дети", проверяем с помощью value_counts()

data['children'] = data['children'].replace(-1, 1)
data['children'] = data['children'].replace(20, 2)
data['children'].value_counts()

# проверим связь между наличием ребенка и долгом

def children_debt(row):
    children = row['children'] 
    debt = row['debt']
    if debt == 1:
        if children < 1:
            return 'нет детей и есть долг'
        if 1 <= children <= 2:
            return 'есть дети и есть долг'
        if children > 2:
            return 'многодетная семья и есть долг'
    if debt == 0:
        if children < 1:
            return 'нет детей и нет долга'
        if 1 <= children <= 2:
            return 'есть дети и нет долга'
        if children > 2:
            return 'многодетная семья и нет долга'
           
data['children_debt'] = data.apply(children_debt, axis = 1)
#print(data.head(10))

   children  days_employed  dob_years education  education_id  \
0         1          -8438         42    высшее             0   
1         1          -4025         36   среднее             1   
2         0          -5623         33   среднее             1   
3         3          -4125         32   среднее             1   
4         0         340266         53   среднее             1   
5         0           -926         27    высшее             0   
6         0          -2879         43    высшее             0   
7         0           -153         50   среднее             1   
8         2          -6930         35    высшее             0   
9         0          -2189         41   среднее             1   

      family_status  family_status_id gender income_type  debt  total_income  \
0   женат / замужем                 0      F   сотрудник     0        253875   
1   женат / замужем                 0      F   сотрудник     0        112080   
2   женат / замужем                 0      M

In [ ]:
# проверим связь между семейным положением и долгом

def family_status_debt(row):
    family_status = row['family_status'] 
    debt = row['debt']
    if debt == 1:
        if family_status == 'женат / замужем':
            return 'семейный человек и есть долг'
        if family_status == 'гражданский брак':
            return 'семейный человек и есть долг'
        if family_status == 'Не женат / не замужем':
            return 'одинокий человек и есть долг'
        if family_status == 'вдовец / вдова':
            return 'одинокий человек и есть долг'
    if debt == 0:  
        if family_status == 'женат / замужем':
            return 'семейный человек и нет долга'
        if family_status == 'гражданский брак':
            return 'семейный человек и нет долга'
        if family_status == 'Не женат / не замужем':
            return 'одинокий человек и нет долга'
        if family_status == 'вдовец / вдова':
            return 'одинокий человек и нет долга'         
data['family_status_debt'] = data.apply(family_status_debt, axis = 1)
#print(data.head(10))

In [16]:
# функция для разделения категорий доходов, для того чтобы потом по ней удобнее строить сводную таблицу
def total_income_type(row):
    total_income = row['total_income']
    if total_income <= 75000:
            return 'низкий доход'
    if total_income <= 150000:
            return 'средний доход'
    if total_income > 150000:
            return 'высокий доход'
data['total_income_type'] = data.apply(total_income_type, axis = 1)


def total_income_debt(row):
    total_income = row['total_income'] 
    debt = row['debt']
    if debt == 1:
        if total_income <= 75000:
            return 'низкий доход и есть долг'
        if total_income <= 150000:
            return 'средний доход и есть долг'
        if total_income > 150000:
            return 'высокий доход и есть долг'
    if debt == 0:
        if total_income <= 75000:
            return 'низкий доход и нет долга'
        if total_income <= 150000:
            return 'средний доход и нет долга'
        if total_income > 150000:
            return 'высокий доход и нет долга'       
data['total_income_debt'] = data.apply(total_income_debt, axis = 1)

#print(data.head(10))

In [17]:
# проверим связь между целью кредита и долгом

def type_purpose_debt(row):
    type_purpose = row['type_purpose']
    debt = row['debt']
    if debt == 1:
        if type_purpose == 'недвижимость':
            return 'недвижимость и есть долг'
        if type_purpose == 'автомобиль':
            return 'автомобиль и есть долг'
        if type_purpose == 'образование':
            return 'образование и есть долг'
        if type_purpose == 'прочие цели':
            return 'прочие цели и есть долг'   
    if debt == 0:    
        if type_purpose == 'недвижимость':
            return 'недвижимость и нет долга'
        if type_purpose == 'автомобиль':
            return 'автомобиль и нет долга'
        if type_purpose == 'образование':
            return 'образование и нет долга'
        if type_purpose == 'прочие цели':
            return 'прочие цели и нет долга'
data['type_purpose_debt'] = data.apply(type_purpose_debt, axis = 1)
print(data.head(10))

   children  days_employed  dob_years education  education_id  \
0         1          -8438         42    высшее             0   
1         1          -4025         36   среднее             1   
2         0          -5623         33   среднее             1   
3         3          -4125         32   среднее             1   
4         0         340266         53   среднее             1   
5         0           -926         27    высшее             0   
6         0          -2879         43    высшее             0   
7         0           -153         50   среднее             1   
8         2          -6930         35    высшее             0   
9         0          -2189         41   среднее             1   

      family_status  family_status_id gender income_type  debt  total_income  \
0   женат / замужем                 0      F   сотрудник     0        253875   
1   женат / замужем                 0      F   сотрудник     0        112080   
2   женат / замужем                 0      M

Мы созали 4 функции для категоризации (влияние различных факторов на возврат долга):
- наличия детей;
- семейное положение;
- доход;
- цель кредита.

## Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [11]:
pivot_children = data.pivot_table(index='children', columns= 'debt', values='days_employed', aggfunc='count')
pivot_children['percent'] = pivot_children[1] / (pivot_children[1] + pivot_children[0]) * 100
print(pivot_children)

debt            0       1   percent
children                           
0         13028.0  1063.0  7.543822
1          4410.0   445.0  9.165808
2          1926.0   202.0  9.492481
3           303.0    27.0  8.181818
4            37.0     4.0  9.756098
5             9.0     NaN       NaN


Наличие детей в среднем на 1.5 % ухудшает шансы на возврат кредита, возможно это связано с тем, что с появлением детей увеличиваются и траты.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [12]:
pivot_family = data.pivot_table(index='family_status', columns= 'debt', values='days_employed', aggfunc='count')
pivot_family['percent'] = pivot_family[1] / (pivot_family[1] + pivot_family[0]) * 100
print(pivot_family)

debt                       0    1   percent
family_status                              
Не женат / не замужем   2536  274  9.750890
в разводе               1110   85  7.112971
вдовец / вдова           896   63  6.569343
гражданский брак        3763  388  9.347145
женат / замужем        11408  931  7.545182


Клиенты, не находящиеся в браке и живущие в гражданском браке, более чем на 2 % хуже платят кредит, чем те кто состоит в браке сейчас или находился ранее. Возможно это связано с тем, что люди принявшие решение о браке более ответственны.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [13]:
pivot_income = data.pivot_table(index='total_income_type', columns= 'debt', values='days_employed', aggfunc='count')
pivot_income['percent'] = pivot_income[1] / (pivot_income[1] + pivot_income[0]) * 100
print(pivot_income)

debt                  0    1   percent
total_income_type                     
высокий доход      9068  763  7.761164
низкий доход       1729  136  7.292225
средний доход      8916  842  8.628817


Клиенты со средним доходом чаще сталкиваются с проблебой возврата кредита (минимум на 1%). Возможно это связано с излишней уверенностью в своих силах. В то время как клиенты с никим доходом являются самыми дисциплинированными, скорее всего они тщательнее продумывают с каких средст будут погашать кредит.

- Как разные цели кредита влияют на его возврат в срок?

In [14]:
pivot_purpose = data.pivot_table(index='type_purpose', columns= 'debt', values='days_employed', aggfunc='count')
pivot_purpose['percent'] = pivot_purpose[1] / (pivot_purpose[1] + pivot_purpose[0]) * 100
print(pivot_purpose)

debt              0    1   percent
type_purpose                      
автомобиль     3903  403  9.359034
недвижимость  10029  782  7.233373
образование    3643  370  9.220035
прочие цели    2138  186  8.003442


Автомобиль и образование лидеры по количеству невозвратов. Возможно клиенты более спонтанно относятся к покупке товаров этих категорий.

## Общий вывод

Получив таблицу с данными, мы привели ее в пригодный для анализа вид. С помощью категоризации объединили данные в категории, вывели в сводные таблицы. Для наглядности посчитали в процентах данные по задолженности. Благодаря этим данным сделали следующие выводы:
- наличие детей в среднем на 1.5 % ухудшает шансы на возврат кредита;
- клиенты, не находящиеся в браке или живущие в гражданском браке, более чем на 2 % хуже платят кредит;
- клиенты со средним доходом минимум на 1 % чаще сталкиваются с проблебой возврата кредита;
- клиенты, берущие кредит на автомобиль и образование на 1.5-2 % чаще не возвращают долг.
С поставленной задачей мы справились. Передаем данные руководству.